In [ ]:
import liora

In [ ]:
df_2019 = liora.carregar_df_2019()
df_2020 =liora.carregar_df_2020()
df_2021 =liora.carregar_df_2021()

## 1. Como se comportam os gastos com cotas parlamentares ao longo do tempo? Existe alguma tendência de aumento ou redução desse custo?

Nesse gráfico...

In [ ]:
fig, ax = plt.subplots()
fig.patch.set_facecolor('white')

analise_por_mes.vlrDocumento.plot(style='-o', ax=ax, figsize=(16, 9), ms=5)

indices = analise_por_mes.index.to_frame()
indices = indices.numMes.astype(str) + '/' + indices.numAno.astype(str)
indices = indices.to_numpy()
indices = utils.formatar_datas_para_rotulo(pd.Series(indices))

min_y, max_y = utils.gerar_limites_eixo_y(
    analise_por_mes.vlrDocumento.to_numpy())
yticks = np.linspace(minimo, maximo, 7)

ax.set_xticks(range(len(indices)))
ax.set_xticklabels(indices, rotation=90)
ax.grid(color=(1, 0.2, 0.5, 0.3), linestyle=':', linewidth=0.25)
ax.set_ylim([min_y, max_y])
ax.set_xlabel('Mês-Ano', fontsize=15)
ax.set_ylabel(
    'Gastos De Todos os Deputados no Período entre 01/2019 e 08/2021', fontsize=15)

ax.set_yticks(yticks)

yticks = [f'R$ {x/1000000}M' for x in yticks]
ax.set_yticklabels(yticks)

valores = analise_por_mes.vlrDocumento.to_numpy()
valores = np.concatenate(([max_y], valores, [max_y]))

direcao_desvio = np.sign(np.diff(valores))
direcao_desvio[[2, 19, 21, 22, 27]] = -1
direcao_desvio[[10, 11, 14, 20, 26, 29, 30]] = 1
desvios = direcao_desvio * (max_y/30)

interate = zip(desvios, analise_por_mes.vlrDocumento.to_numpy())


for eixo_x, (desvio_y, valor) in enumerate(interate):
    valor_sem_resto = valor - (valor % (base/100))
    coordenadas = (eixo_x, valor_sem_resto + desvio_y)
    ax.annotate(f'{(valor_sem_resto/base):4.2f}M',
                coordenadas, fontsize=8, ha='center')

plt.savefig('Gasto Total sem Barra Vertical.png', dpi=150, bbox_inches='tight')
plt.bar([23, 15], [max_y, max_y], color=['#ffcccc', '#ccffcc'])
plt.savefig('Gasto Total com Barra Vertical.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Quais são as categorias de despesas mais onerosas dentre os recursos destinados às cotas parlamentares?

In [ ]:
liora.categoria_despesa_onerosa(df_2019)
liora.categoria_despesa_onerosa(df_2020)
liora.categoria_despesa_onerosa(df_2021)

## 5. Quais são os trechos de viagem mais recorrentes entre as despesas de viagem?

In [ ]:
liora.trecho_viagem_recorrente(df_2019)
liora.trecho_viagem_recorrente(df_2020)
liora.trecho_viagem_recorrente(df_2021)

In [ ]:
import odilon

## 7. Qual é o valor total de gastos por estado?

In [ ]:
odilon.valor_total_de_gasto_estado(df_2019)
odilon.valor_total_de_gasto_estado(df_2020)
odilon.valor_total_de_gasto_estado(df_2021)


## 8. Quais estados possuem maior numero de representantes ?

In [ ]:
odilon.quantidade_representantes_por_estado(df_2019)
odilon.quantidade_representantes_por_estado(df_2020)
odilon.quantidade_representantes_por_estado(df_2021)


## 9. Há alguma correlação entre a quantidade de parlamentares por estado e o valor total dos gastos ?

Nesses gráficos...

In [ ]:
odilon.gráfico_correlacao_parlamentar_gastoporestado(df_2019)
odilon.gráfico_correlacao_parlamentar_gastoporestado(df_2020)
odilon.gráfico_correlacao_parlamentar_gastoporestado(df_2021)